In [1]:
import pandas as pd

In [2]:
eng = pd.read_csv('premier_league.csv').sort_values(by=['year'], ascending=False)
ita = pd.read_csv('serie_a.csv').sort_values(by=['year'], ascending=False)
rus = pd.read_csv('premier_liga.csv').sort_values(by=['year'], ascending=False)
por = pd.read_csv('liga_nos.csv').sort_values(by=['year'], ascending=False)
net = pd.read_csv('eredivisie.csv').sort_values(by=['year'], ascending=False)
ger = pd.read_csv('bundesliga.csv').sort_values(by=['year'], ascending=False)
spa = pd.read_csv('primera_division.csv').sort_values(by=['year'], ascending=False)
fra = pd.read_csv('ligue_1.csv').sort_values(by=['year'], ascending=False)

# Concatenate dataframes
concatenated_df = pd.concat([eng, ita, rus, por, net, ger, spa, fra], ignore_index=True)

# Filter rows with year == 2022 and fee not containing 'loan' or 'Loan'
concatenated_df['fee'] = concatenated_df['fee'].astype(str)  # Convert 'fee' column to string
concatenated_df = concatenated_df[(concatenated_df['year'] == 2022)]

# Reset index
concatenated_df.reset_index(drop=True, inplace=True)

concatenated_df['league_name'] = concatenated_df['league_name'].replace({'1 Bundesliga': 'Bundesliga', 'Primera Division': 'La Liga'})

In [3]:
def extract_club(row):
    if row['transfer_movement'] == 'in':
        return row['club_name']
    elif row['transfer_movement'] == 'out':
        return row['club_involved_name']
    else:
        return None  # You might want to handle other cases as needed

# Apply the function to create the new 'Club' column
concatenated_df['Club'] = concatenated_df.apply(extract_club, axis=1)

# Drop the 'club_name' and 'club_involved_name' columns
concatenated_df = concatenated_df.drop(['club_name', 'club_involved_name','year','fee_cleaned','position','country','transfer_movement'], axis=1)

In [4]:
concatenated_df

,player_name,age,fee,transfer_period,league_name,season,Club
0,Matija Sarkic,25.0,loan transfer,Winter,Premier League,2022/2023,Stoke City
1,Steven Sessegnon,22.0,loan transfer,Summer,Premier League,2022/2023,Charlton
2,Rasmus Kristensen,24.0,€13.00m,Summer,Premier League,2022/2023,Leeds United
3,Tyler Adams,23.0,€17.00m,Summer,Premier League,2022/2023,Leeds United
4,Luis Sinisterra,23.0,€25.00m,Summer,Premier League,2022/2023,Leeds United
...,...,...,...,...,...,...,...
6157,Jean-Claude Billong,28.0,free transfer,Summer,Ligue 1,2022/2023,CFR Cluj
6158,Akim Zedadka,27.0,free transfer,Summer,Ligue 1,2022/2023,LOSC Lille
6159,Fred Gnalega,20.0,loan transfer,Summer,Ligue 1,2022/2023,Chamalières
6160,Arthur Desmas,28.0,?,Summer,Ligue 1,2022/2023,AC Le Havre


In [5]:
from fuzzywuzzy import fuzz

def team_lookup(name, name_list):
    max_score = 0
    name_to_ret = ''
    for team in name_list:
        temp_score = fuzz.ratio(name, team)
        if temp_score > max_score:
            max_score = temp_score
            name_to_ret = team
        
    return name_to_ret

In [6]:
pl = ['Liverpool','R.F.C. Seraing',"M'Gladbach",'Torpedo Moscow','Betis','Villarreal','Manchester Utd',
     'Famalicão','Rangers','Hibernian','Bayern Munich','Hearts','Monza','Aston Villa','Porto','Marítimo',
     'CSKA Moscow','Sevilla','Nice','Antwerp','Inter','Hertha BSC','Tottenham','Cádiz','Real Madrid','Ajaccio','Lyon',
     'Barcelona','Sassuolo','Salernitana','Milan','Spezia','Sampdoria','Auxerre','Rennes','Casa Pia',
     'Anderlecht','Fiorentina','Ajax','Paris S-G','Ural','Aberdeen','Gil Vicente FC','Strasbourg','Lens',
     'Ross County','Zenit','Chelsea','Wolves','Cremonese','West Ham','Twente','Rayo Vallecano','RKC Waalwijk',
     "Sparta R'dam",'Zulte Waregem','Bologna','Southampton','Lorient','Lille','Westerlo','Everton','Athletic Club',
     'NEC Nijmegen','Augsburg','Fortuna Sittard','Atlético Madrid','Dundee United','Dortmund','Real Sociedad',
     'Wolfsburg','Schalke 04','Kilmarnock','Dynamo Mosc','Marseille','Gent','Lecce','Rio Ave','Sint-Truiden',
     'Eint Frankfurt','Juventus','Celtic','Bochum','St Mirren','Crystal Palace','Leicester City','Oostende',
     'Empoli','Freiburg','Brighton','Hellas Verona','Sporting CP','Lazio','Roma',"Nott'ham Forest",'St Johnstone',
     'Sochi','Nantes','Osasuna','Hoffenheim','Loko Moscow','Udinese','Montpellier','Valencia','Club Brugge',
     'Troyes','Mallorca','Valladolid','Livingston','PSV Eindhoven','Fulham','Portimonense','RB Leipzig','Celta Vigo',
     'Chaves','Benfica','Elche','Nizhny Novgorod','Utrecht','Charleroi','Orenburg','Werder Bremen','Feyenoord',
     'Girona','Almería','Spartak Moscow','Clermont Foot','Leverkusen','Cambuur','Motherwell','Bournemouth',
     'Emmen','Angers','Vitória','Mechelen','Braga','Leeds United','Arouca','Vizela','Atalanta','Union SG',
     'Brentford','Newcastle Utd','Union Berlin','Getafe','Vitesse','Stuttgart','OH Leuven','Espanyol','Monaco',
     'Napoli','Mainz 05','Go Ahead Eag','Manchester City','Fakel Voronezh','Torino','Heerenveen','Arsenal',
     'AZ Alkmaar','Köln','Cercle Brugge','Kortrijk','Akhmat Grozny','Standard Liège','Boavista','Brest','Santa Clara',
     'Toulouse','Estoril','Paços','Eupen','FC Khimki','Groningen','Samara','Volendam','Excelsior','Krasnodar',
     'Rostov','Reims','Genk','no_team']

pn = list(concatenated_df['Club'].unique())

new_pl = pl
new_pn = pn.copy()
for i in range(len(pn) - 1, -1, -1):
    name = pn[i]
    np = team_lookup(name, pl)
    if name == np:
        del pn[i]
        
replacements = {}
for name in pn:
    np2 = team_lookup(name, pl)
    print("replacements['{}'] = '{}'".format(name, np2))
    replacements[name] = 'no_team'

replacements['Stoke City'] = 'Leicester City'
replacements['Charlton'] = 'Charleroi'
replacements['Leeds'] = 'Lens'
replacements['Without Club'] = 'Athletic Club'
replacements['Middlesbrough'] = 'Wolfsburg'
replacements['Alanyaspor'] = 'Atalanta'
replacements['OGC Nice'] = 'Nice'
replacements['HNK Gorica'] = 'Mallorca'
replacements['AC Le Havre'] = 'OH Leuven'
replacements['Wycombe'] = 'Wolves'
replacements['Hull City'] = 'Leicester City'
replacements['SSC Napoli'] = 'Napoli'
replacements['Millwall'] = 'Milan'
replacements['UD Ibiza'] = 'Vizela'
replacements['Ipswich'] = 'Nice'
replacements['Man City'] = 'Manchester City'
replacements['Fulham FC'] = 'Fulham'
replacements['Everton FC'] = 'Everton'
replacements['Besiktas'] = 'Betis'
replacements['Spurs'] = 'Sampdoria'
replacements['Trabzonspor'] = 'Krasnodar'
replacements['St. Johnstone'] = 'St Johnstone'
replacements['RWDM'] = 'Real Madrid'
replacements['Nottm Forest'] = 'Nott'ham Forest'
replacements['Botafogo'] = 'Bologna'
replacement

replacements['PAS Lamia'] = 'Sampdoria'
replacements['FK Orenburg'] = 'Orenburg'
replacements['M. Bnei Reineh'] = 'Union Berlin'
replacements['KAS Eupen'] = 'Eupen'
replacements['Irtysh Omsk'] = 'Hertha BSC'
replacements['FC Olympic'] = 'FC Khimki'
replacements['Rodina 2 Moscow'] = 'Torpedo Moscow'
replacements['Navbahor'] = 'M'Gladbach'
replacements['FK Khimki'] = 'FC Khimki'
replacements['SV Sandhausen'] = 'PSV Eindhoven'
replacements['Lech Poznan'] = 'Hellas Verona'
replacements['Pyunik Yerevan'] = 'Union Berlin'
replacements['Djurgården'] = 'Auxerre'
replacements['FK Minsk'] = 'St Mirren'
replacements['Tyumen'] = 'Emmen'
replacements['Legia Warszawa'] = 'Hearts'
replacements['Ordabasy'] = 'Braga'
replacements['Shinnik'] = 'Sochi'
replacements['Ural 2'] = 'Ural'
replacements['Ban'] = 'Barcelona'
replacements['Leningradets'] = 'Rennes'
replacements['Loko-Kazanka M.'] = 'Loko Moscow'
replacements['Kairat Almaty'] = 'Kilmarnock'
replacements['Chelyabinsk'] = 'Chelsea'
replacements['Tor

replacements['Le Puy Foot'] = 'Clermont Foot'
replacements['FC Martigues'] = 'Marítimo'
replacements['Al-Arabi SC'] = 'Hertha BSC'
replacements['AS Nancy'] = 'Nantes'
replacements['US Avranches'] = 'Manchester Utd'
replacements['Kilmarnock FC'] = 'Kilmarnock'
replacements['Stade Rennais FC'] = 'Rennes'
replacements['Western Sydney'] = 'Westerlo'
replacements['Burnley U21'] = 'Bournemouth'
replacements['Paris 13 Atl.'] = 'Paris S-G'
replacements['Shakhtar D.'] = 'Schalke 04'
replacements['Melbourne'] = 'Fakel Voronezh'
replacements['FC Sète 34'] = 'R.F.C. Seraing'
replacements['Concarneau'] = 'Bournemouth'
replacements['FC Toulouse'] = 'Toulouse'
replacements['FC Biel-Bienne'] = 'NEC Nijmegen'
replacements['Grenoble'] = 'Arsenal'
replacements['Clermont Foot 63'] = 'Clermont Foot'
replacements['Quevilly Rouen'] = 'Sevilla'
replacements['Chamois Niort'] = 'Nott'ham Forest'
replacements['Stabæk Fotball'] = 'Spartak Moscow'
replacements['FC Koper'] = 'FC Khimki'
replacements['Le Mans FC'] =

In [7]:
replacements['SSC Napoli'] = 'Napoli'
replacements['Man City'] = 'Manchester City'
replacements['Everton FC'] = 'Everton'
replacements['Spurs'] = 'Tottenham'
replacements['Nottm Forest'] = "Nott'ham Forest"
replacements['Fulham FC'] = 'Fulham'
replacements["Bor. M'gladbach"] = "M'Gladbach"
replacements['Manchester United'] = 'Manchester Utd'
replacements['Liverpool FC'] = 'Liverpool'
replacements['OGC Nice'] = 'Nice'
replacements['FC Porto'] = 'Porto'
replacements['AC Milan'] = 'Milan'
replacements['AFC Bournemouth'] = 'Bournemouth'
replacements['Newcastle'] = 'Newcastle Utd'
replacements['Brentford FC'] = 'Brentford'
replacements['Arsenal FC'] = 'Arsenal'
replacements['VfB Stuttgart'] = 'Stuttgart'
replacements['Olympique Lyon'] = 'Lyon'
replacements['Chelsea FC'] = 'Chelsea'
replacements['Man Utd'] = 'Manchester Utd'
replacements['Real Valladolid'] = 'Valladolid'
replacements['Brighton & Hove Albion'] = 'Brighton'
replacements['AJ Auxerre'] = 'Auxerre'
replacements['R. Strasbourg'] = 'Strasbourg'
replacements['Southampton FC'] = 'Southampton'
replacements['West Ham United'] = 'West Ham'
replacements['Wolverhampton Wanderers'] = 'Wolves'
replacements['Nottingham Forest'] = "Nott'ham Forest"
replacements['Newcastle United'] = 'Newcastle Utd'
replacements['Elche CF'] = 'Elche'
replacements['AS Roma'] = 'Roma'
replacements['Tottenham Hotspur'] = 'Tottenham'
replacements['Angers SCO'] = 'Angers'
replacements['Torino FC'] = 'Torino'
replacements['US Sassuolo'] = 'Sassuolo'
replacements['Udinese Calcio'] = 'Udinese'
replacements['Atalanta BC'] = 'Atalanta'
replacements['VfL Wolfsburg'] = 'Wolfsburg'
replacements['Bologna FC 1909'] = 'Bologna'
replacements['Stade Rennais'] = 'Rennes'
replacements['FC Empoli'] = 'Empoli'
replacements['Spezia Calcio'] = 'Spezia'
replacements['LOSC Lille'] = 'Lille'
replacements['Paris SG'] = 'Paris S-G'
replacements['Juventus FC'] = 'Juventus'
replacements['Inter Milan'] = 'Inter'
replacements['RFC Seraing'] = 'R.F.C. Seraing'
replacements['E. Frankfurt'] = 'Eint Frankfurt'
replacements['Stade Reims'] = 'Reims'
replacements['SS Lazio'] = 'Lazio'
replacements['RCD Mallorca'] = 'Mallorca'
replacements['ACF Fiorentina'] = 'Fiorentina'
replacements['Real Betis'] = 'Betis'
replacements['Go Ahead Eagles'] = 'Go Ahead Eag'
replacements['RSC Anderlecht'] = 'Anderlecht'
replacements['AC Monza'] = 'Monza'
replacements['US Salernitana 1919'] = 'Salernitana'
replacements['US Lecce'] = 'Lecce'
replacements['FC Lorient'] = 'Lorient'
replacements['UC Sampdoria'] = 'Sampdoria'
replacements['FC Schalke 04'] = 'Schalke 04'
replacements['US Cremonese'] = 'Cremonese'
replacements['FK Rostov'] = 'Rostov'
replacements['Krylya Sovetov Samara'] = 'Samara'
replacements['FC Pari Nizhniy Novgorod'] = 'Nizhny Novgorod'
replacements['Khimki'] = 'FC Khimki'
replacements['Zenit S-Pb'] = 'Zenit'
replacements['KS Samara'] = 'Samara'
replacements['Dynamo Moscow'] = 'Dynamo Mosc'
replacements['FK Krasnodar'] = 'Krasnodar'
replacements['FC Sochi'] = 'Sochi'
replacements['Gil Vicente'] = 'Gil Vicente FC'
replacements['Zenit St. Petersburg'] = 'Zenit'
replacements['Lokomotiv Moscow'] = 'Loko Moscow'
replacements['Sparta R.'] = "Sparta R'dam"
replacements['FK Orenburg'] = 'Orenburg'
replacements['KAS Eupen'] = 'Eupen'
replacements['FK Khimki'] = 'FC Khimki'
replacements['FC Famalicão'] = 'Famalicão'
replacements['Rio Ave FC'] = 'Rio Ave'
replacements['GD Estoril Praia'] = 'Estoril'
replacements['CD Santa Clara'] = 'Santa Clara'
replacements['UD Almería'] = 'Almería'
replacements['CS Marítimo'] = 'Marítimo'
replacements['SL Benfica'] = 'Benfica'
replacements['Celta de Vigo'] = 'Celta Vigo'
replacements['Vitória Guimarães SC'] = 'Vitória'
replacements['SC Braga'] = 'Braga'
replacements['TSG Hoffenheim'] = 'Hoffenheim'
replacements['GD Chaves'] = 'Chaves'
replacements['Portimonense SC'] = 'Portimonense'
replacements['FC Arouca'] = 'Arouca'
replacements['Casa Pia AC'] = 'Casa Pia'
replacements['FC Vizela'] = 'Vizela'
replacements['Estoril Praia'] = 'Estoril'
replacements['Boavista FC'] = 'Boavista'
replacements['Vitória Setúbal'] = 'Vitória'
replacements['FC Utrecht'] = 'Utrecht'
replacements['FC Groningen'] = 'Groningen'
replacements['SC Heerenveen'] = 'Heerenveen'
replacements['SC Cambuur'] = 'Cambuur'
replacements['SC Cambuur-Leeuwarden'] = 'Cambuur'
replacements['FC Emmen'] = 'Emmen'
replacements['Bor. Dortmund'] = 'Dortmund'
replacements['Twente FC'] = 'Twente'
replacements['Feyenoord Rotterdam'] = 'Feyenoord'
replacements['SC Cambuur Leeuwarden'] = 'Cambuur'
replacements['FC Volendam'] = 'Volendam'
replacements['Twente Enschede FC'] = 'Twente'
replacements['Sparta Rotterdam'] = "Sparta R'dam"
replacements['Vitesse Arnhem'] = 'Vitesse'
replacements['SC Freiburg'] = 'Freiburg'
replacements['TSG 1899 Hoffenheim'] = 'Hoffenheim'
replacements['Borussia Mönchengladbach'] = "M'Gladbach"
replacements['VfL Bochum'] = 'Bochum'
replacements['1. FC Köln'] = 'Köln'
replacements['1.FSV Mainz 05'] = 'Mainz 05'
replacements['KV Mechelen'] = 'Mechelen'
replacements['FC Augsburg'] = 'Augsburg'
replacements['Viktoria Köln'] = 'Vitória'
replacements['Eintracht Frankfurt'] = 'Eint Frankfurt'
replacements['Bayer 04 Leverkusen'] = 'Leverkusen'
replacements['Sevilla FC'] = 'Sevilla'
replacements['Borussia Dortmund'] = 'Dortmund'
replacements['1.FC Union Berlin'] = 'Union Berlin'
replacements['SV Werder Bremen'] = 'Werder Bremen'
replacements['FC Nantes'] = 'Nantes'
replacements['KAA Gent'] = 'Gent'
replacements['Royal Antwerp'] = 'Antwerp'
replacements['CA Osasuna'] = 'Osasuna'
replacements['Athletic Bilbao'] = 'Athletic Club'
replacements['Valencia CF'] = 'Valencia'
replacements['Atlético de Madrid'] = 'Atlético Madrid'
replacements['Cádiz CF'] = 'Cádiz'
replacements['FC Barcelona'] = 'Barcelona'
replacements['Villarreal CF'] = 'Villarreal'
replacements['Girona FC'] = 'Girona'
replacements['Real Valladolid CF'] = 'Valladolid'
replacements['Getafe CF'] = 'Getafe'
replacements['RC Lens'] = 'Lens'
replacements['Pau FC'] = 'Paris S-G'
replacements['Montpellier HSC'] = 'Montpellier'
replacements['Leicester'] = 'Leicester City'
replacements['AC Ajaccio'] = 'Ajaccio'
replacements['Paris FC'] = 'Paris S-G'
replacements['Olympique Marseille'] = 'Marseille'
replacements['AS Monaco'] = 'Monaco'
replacements['Paris Saint-Germain'] = 'Paris S-G'
replacements['RC Strasbourg Alsace'] = 'Strasbourg'
replacements['Stade Rennais FC'] = 'Rennes'
replacements['Athletic'] = 'Athletic Club'
replacements['FC Toulouse'] = 'Toulouse'
replacements['Clermont Foot 63'] = 'Clermont Foot'
replacements['B. Leverkusen'] = 'Leverkusen'
replacements['ESTAC Troyes'] = 'Troyes'
replacements['Paços Ferreira'] = 'Paços'
replacements['Leeds'] = 'Leeds United'

In [8]:
concatenated_df['Squad'] = concatenated_df['Club'].replace(replacements)

# Drop rows where 'Club' column is "no_team"
concatenated_df = concatenated_df[concatenated_df['Squad'] != "no_team"]

concatenated_df['player_code'] = concatenated_df['player_name'].str.lower().str.replace(' ','-')

replacements = {
    'à': 'a', 'á': 'a', 'â': 'a', 'ä': 'a', 'ǎ': 'a', 'æ': 'a', 'ã': 'a', 'å': 'a', 'ā': 'a',
    'è': 'e', 'é': 'e', 'ê': 'e', 'ë': 'e', 'ě': 'e', 'ẽ': 'e', 'ē': 'e', 'ė': 'e', 'ę': 'e',
    'ì': 'i', 'í': 'i', 'î': 'i', 'ï': 'i', 'ǐ': 'i', 'ĩ': 'i', 'ī': 'i', 'ı': 'i', 'į': 'i',
    'ò': 'o', 'ó': 'o', 'ô': 'o', 'ö': 'o', 'ǒ': 'o', 'œ': 'o', 'ø': 'o', 'õ': 'o', 'ō': 'o',
    'ù': 'u', 'ú': 'u', 'û': 'u', 'ü': 'u', 'ǔ': 'u', 'ũ': 'u', 'ū': 'u', 'ű': 'u', 'ů': 'u',
    'ğ':'g','ġ':'g','Ğ':'G','Ġ':'G','ç':'c','ć':'c','č':'c','ċ':'c','ł':'l','ļ':'l','ľ':'l',
    'ß':'s','ş':'s','ș':'s','ś':'s','š':'s','ķ':'k','ñ':'n','ń':'n','ņ':'n','ň':'n','ŵ':'w',
    'ź':'z','ž':'z','ż':'z','þ':'b','đ':'d',"'":'',"’":'','ý':'y','i̇':'i','ă':'a','ř':'r'
}

for key in replacements:
    concatenated_df['player_code'] = concatenated_df['player_code'].str.replace(key, replacements[key])
    
concatenated_df

/var/folders/08/f0gkj3cx58d1cxy80qf50nhw0000gn/T/ipykernel_2565/3481190945.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df['player_code'] = concatenated_df['player_name'].str.lower().str.replace(' ','-')
/var/folders/08/f0gkj3cx58d1cxy80qf50nhw0000gn/T/ipykernel_2565/3481190945.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df['player_code'] = concatenated_df['player_code'].str.replace(key, replacements[key])


,player_name,age,fee,transfer_period,league_name,season,Club,Squad,player_code
2,Rasmus Kristensen,24.0,€13.00m,Summer,Premier League,2022/2023,Leeds United,Leeds United,rasmus-kristensen
3,Tyler Adams,23.0,€17.00m,Summer,Premier League,2022/2023,Leeds United,Leeds United,tyler-adams
4,Luis Sinisterra,23.0,€25.00m,Summer,Premier League,2022/2023,Leeds United,Leeds United,luis-sinisterra
5,Brenden Aaronson,21.0,€32.84m,Summer,Premier League,2022/2023,Leeds United,Leeds United,brenden-aaronson
6,Daniel James,25.0,"End of loanMay 31, 2023",Summer,Premier League,2022/2023,Leeds,Leeds United,daniel-james
...,...,...,...,...,...,...,...,...,...
6147,Lucas Da Cunha,21.0,"End of loanJun 30, 2022",Summer,Ligue 1,2022/2023,OGC Nice,Nice,lucas-da-cunha
6151,Veljko Birmancevic,24.0,€4.10m,Summer,Ligue 1,2022/2023,FC Toulouse,Toulouse,veljko-birmancevic
6152,Kjetil Haug,24.0,?,Summer,Ligue 1,2022/2023,FC Toulouse,Toulouse,kjetil-haug
6156,Oriol Busquets,23.0,free transfer,Summer,Ligue 1,2022/2023,Arouca,Arouca,oriol-busquets


In [9]:
players = pd.read_csv('player_data_2023.csv')
pl = list(players['player_code'].unique())
pn = list(concatenated_df['player_code'])

new_pl = pl
new_pn = pn.copy()
replacements = {}
for i in range(len(pn) - 1, -1, -1):
    name = pn[i]
    np = team_lookup(name, pl)
#     squad_value = concatenated_df[concatenated_df['player_code'] == name]['Squad'].values[0]
#     check = players[players['player_code'] == np]['Squad'].values[0]

    if name == np[:-2]:
        replacements[name] = np
        del pn[i]
        
for name in pn:
    np2 = team_lookup(name, pl)
    replacements[name] = 'na'

In [10]:
replacements['wilfried-gnonto'] = 'degnand-gnonto03'
replacements['andre-zambo-anguissa'] = 'andre-frank-zambo-anguissa95'
replacements['idrissa-gueye'] = 'idrissa-gana-gueye89'
replacements['leo-ostigard'] = 'leo-skiri-ostigard99'
replacements['mykhaylo-mudryk'] = 'mykhailo-mudryk01'
replacements['ilya-zabarnyi'] = 'illia-zabarnyi02'
replacements['victor-kristiansen'] = 'victor-bernth-kristiansen02'
replacements['hee-chan-hwang'] = 'hwang-hee-chan96'
replacements['bremer'] = 'gleison-bremer97'
replacements['emil-konradsen-ceide'] = 'emil-ceide01'
replacements['milan-djuric'] = 'milan-duric90'
replacements['juan-cabal'] = 'juan-david-cabal01'
replacements['charalampos-lykogiannis'] = 'charalambos-lykogiannis93'
replacements['duccio-degli-innocenti'] = 'duccio-deglinnocenti03'
replacements['filip-djuricic'] = 'filip-duricic92'
replacements['min-jae-kim'] = 'kim-min-jae96'
replacements['andreaw-gravillon'] = 'andrew-gravillon98'
replacements['bartlomiej-dragowski'] = 'bartlomiej-drągowski97'
replacements['vivaldo'] = 'vivaldo-semedo05'
replacements['emanuel-aiwu'] = 'emanuel-aiwum00'
replacements['maissa-ndiaye'] = 'moussa-ndiaye02'
replacements['christopher-martins-pereira'] = 'christopher-martins97'
replacements['roman-evgenjev'] = 'roman-yevgenyev99'
replacements['georgiy-zotov'] = 'georgi-zotov90'
replacements['sergey-bozhin'] = 'sergei-bozhin94'
replacements['ilya-zhigulev'] = 'ilya-zhigulyov96'
replacements['vitaliy-shitov'] = 'vitaly-shitov03'
replacements['zelimkhan-bakaev'] = 'zelimkhan-bakayev96'
replacements['vitaliy-gudiev'] = 'vitali-gudiyev95'
replacements['nicolas-moumi-ngamaleu'] = 'moumi-ngamaleu94'
replacements['ifeanyi-david-nduka'] = 'ifeanyi-nduka03'
replacements['artem-meshchaninov'] = 'artyom-meshchaninov96'
replacements['yuriy-gazinskiy'] = 'yuri-gazinskiy89'
replacements['artur-chernyi'] = 'artur-chyorny00'
replacements['ivan-ignatjev'] = 'ivan-ignatyev99'
replacements['vladimir-iljin'] = 'vladimir-ilyin92'
replacements['ivan-oleynikov'] = 'ivan-oleinikov98'
replacements['yuri-zhuravlev'] = 'yuri-zhuravlyov96'
replacements['nikolay-rasskazov'] = 'nikolai-rasskazov98'
replacements['aleksandr-zuev'] = 'aleksandr-zuyev96'
replacements['bi-nene-junior-gbamble'] = 'nene-gbamble02'
replacements['evgeniy-shlyakov'] = 'yevgeni-shlyakov91'
replacements['martin-aguirregabiria'] = 'martin-agirregabiria96'
replacements['lucho-vega'] = 'luciano-vega99'
replacements['benchimol'] = 'gilson-benchimol01'
replacements['volnei'] = 'volnei-feltes00'
replacements['pedro-silva'] = 'joao-pedro-silva99'
replacements['joao-afonso'] = 'joao-afonso-crispim95'
replacements['john-anthony-brooks'] = 'john-brooks93'
replacements['jeremiah-st.-juste'] = 'jerry-st.-juste96'
replacements['gil-dias'] = 'gil-bastiao-dias96'
replacements['alexsandro'] = 'alex-sandro91'
replacements['ponck'] = 'carlos-ponck95'
replacements['chermiti'] = 'youssef-chermiti04'
replacements['paulo-estrela'] = 'paulo-estrela-moreira99'
replacements['zie-ouattara'] = 'aziz-ouattara01'
replacements['yan-eteki'] = 'yan-brice-eteki97'
replacements['leo-bolgado'] = 'bolgado98'
replacements['guilherme'] = 'guilherme-silva00'
replacements['hernani'] = 'hernani-fortes91'
replacements['willum-thor-willumsson'] = 'willum-bor-willumsson98'
replacements['oliver-valaker-edvardsen'] = 'oliver-edvardsen99'
replacements['tunahan-tasci'] = 'muhammet-tunahan-tasci02'
replacements['isak-dybvik-maatta'] = 'isak-maatta01'
replacements['rick-meissen'] = 'rick-meijssen02'
replacements['konstantinos-stafylidis'] = 'kostas-stafylidis93'
replacements['daniel-kofi-kyereh'] = 'daniel-kyereh96'
replacements['dikeni-salifou'] = 'dikeni-rafid-salifou03'
replacements['dion-beljo'] = 'dion-drena-beljo02'
replacements['juan-jose-perea'] = 'juan-perea00'
replacements['laurin-ulrich'] = 'lauren-ulrich05'
replacements['nicolas-fernandez-mercau'] = 'nicolas-ezequiel-fernandez00'
replacements['adrian-embarba'] = 'adri-embarba92'
replacements['alejandro-pozo'] = 'alejandro-pozo-pozo99'
replacements['robert-navarro'] = 'roberto-navarro02'
replacements['memphis-depay'] = 'memphis94'
replacements['viktor-tsygankov'] = 'viktor-tsyhankov97'
replacements['pape-cheikh'] = 'pape-cheikh-diop97'
replacements['leo-roman'] = 'leonardo-roman00'
replacements['momo-mbaye'] = 'mamadou-mbaye98'
replacements['srdjan-babic'] = 'srdan-babic96'
replacements['mohamed-bayo'] = 'mohamed-lamine-bayo98'
replacements['ilyes-chetti'] = 'ilyes-cheti95'
replacements['nathan-bitumazala'] = 'nathan-bitu-mazala02'
replacements['warren-kamanzi'] = 'waren-kamanzi00'
replacements['amar-abdirahman-ahmed'] = 'amar-ahmed04'
replacements['kays-ruiz-atil'] = 'kays-ruiz02'

In [11]:
concatenated_df['player_code'] = concatenated_df['player_code'].replace(replacements)

# Drop rows where 'player_code' column is "na"
concatenated_df = concatenated_df[concatenated_df['player_code'] != "na"]
concatenated_df

/var/folders/08/f0gkj3cx58d1cxy80qf50nhw0000gn/T/ipykernel_2565/3477425929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df['player_code'] = concatenated_df['player_code'].replace(replacements)


,player_name,age,fee,transfer_period,league_name,season,Club,Squad,player_code
2,Rasmus Kristensen,24.0,€13.00m,Summer,Premier League,2022/2023,Leeds United,Leeds United,rasmus-kristensen97
3,Tyler Adams,23.0,€17.00m,Summer,Premier League,2022/2023,Leeds United,Leeds United,tyler-adams99
4,Luis Sinisterra,23.0,€25.00m,Summer,Premier League,2022/2023,Leeds United,Leeds United,luis-sinisterra99
5,Brenden Aaronson,21.0,€32.84m,Summer,Premier League,2022/2023,Leeds United,Leeds United,brenden-aaronson00
6,Daniel James,25.0,"End of loanMay 31, 2023",Summer,Premier League,2022/2023,Leeds,Leeds United,daniel-james97
...,...,...,...,...,...,...,...,...,...
6144,Thijs Dallinga,21.0,€2.50m,Summer,Ligue 1,2022/2023,FC Toulouse,Toulouse,thijs-dallinga00
6145,Theocharis Tsingaras,22.0,Loan fee:€300Th.,Summer,Ligue 1,2022/2023,FC Toulouse,Toulouse,theocharis-tsingaras00
6151,Veljko Birmancevic,24.0,€4.10m,Summer,Ligue 1,2022/2023,FC Toulouse,Toulouse,veljko-birmancevic98
6156,Oriol Busquets,23.0,free transfer,Summer,Ligue 1,2022/2023,Arouca,Arouca,oriol-busquets99


In [12]:
concatenated_df[concatenated_df['player_code'].str.contains('bereszynski')]

,player_name,age,fee,transfer_period,league_name,season,Club,Squad,player_code
1292,Bartosz Bereszynski,30.0,loan transfer,Winter,Serie A,2022/2023,SSC Napoli,Napoli,bartosz-bereszynski92
1428,Bartosz Bereszynski,30.0,loan transfer,Winter,Serie A,2022/2023,SSC Napoli,Napoli,bartosz-bereszynski92


In [13]:
# Define a function to convert the fee string to numerical value
def convert_fee_to_numeric(fee):
    # Remove the first character if it's '€'
    fee = fee.replace('€', '').replace('Loan fee:', '')

    # Check if the string contains 'm' for million
    if 'm' in fee:
        # Remove 'm' and convert to float, then multiply by 1 million
        return float(fee.replace('m', '')) * 1000000
    # Check if the string contains 'Th.' for thousand
    elif 'Th.' in fee:
        # Remove 'Th.' and convert to float, then multiply by 1 thousand
        return float(fee.replace('Th.', '')) * 1000
    else:
        # If neither 'm' nor 'Th.' is present, return the original fee value
        return 0

# Apply the function to convert the 'fee' column to numerical values
concatenated_df['fee'] = concatenated_df['fee'].apply(convert_fee_to_numeric)
concatenated_df = concatenated_df.drop(['Club','league_name'],axis=1)
concatenated_df

/var/folders/08/f0gkj3cx58d1cxy80qf50nhw0000gn/T/ipykernel_2565/1598831488.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  concatenated_df['fee'] = concatenated_df['fee'].apply(convert_fee_to_numeric)


,player_name,age,fee,transfer_period,season,Squad,player_code
2,Rasmus Kristensen,24.0,13000000.0,Summer,2022/2023,Leeds United,rasmus-kristensen97
3,Tyler Adams,23.0,17000000.0,Summer,2022/2023,Leeds United,tyler-adams99
4,Luis Sinisterra,23.0,25000000.0,Summer,2022/2023,Leeds United,luis-sinisterra99
5,Brenden Aaronson,21.0,32840000.0,Summer,2022/2023,Leeds United,brenden-aaronson00
6,Daniel James,25.0,0.0,Summer,2022/2023,Leeds United,daniel-james97
...,...,...,...,...,...,...,...
6144,Thijs Dallinga,21.0,2500000.0,Summer,2022/2023,Toulouse,thijs-dallinga00
6145,Theocharis Tsingaras,22.0,300000.0,Summer,2022/2023,Toulouse,theocharis-tsingaras00
6151,Veljko Birmancevic,24.0,4100000.0,Summer,2022/2023,Toulouse,veljko-birmancevic98
6156,Oriol Busquets,23.0,0.0,Summer,2022/2023,Arouca,oriol-busquets99


In [14]:
concatenated_df.columns

Index(['player_name', 'age', 'fee', 'transfer_period', 'season', 'Squad',
       'player_code'],
      dtype='object')

In [15]:
concatenated_df[concatenated_df['player_code'].str.contains('adams')]

,player_name,age,fee,transfer_period,season,Squad,player_code
3,Tyler Adams,23.0,17000000.0,Summer,2022/2023,Leeds United,tyler-adams99
4260,Tyler Adams,23.0,17000000.0,Summer,2022/2023,Leeds United,tyler-adams99


In [16]:
concatenated_df = concatenated_df.sort_values('fee', ascending=False).drop_duplicates(['player_code']).sort_index()
concatenated_df[concatenated_df.duplicated(['player_code'], keep=False)]

,player_name,age,fee,transfer_period,season,Squad,player_code


In [17]:
concatenated_df.to_csv('2022_transfers.csv')